In [107]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# define encoder in pytorch
class Encoder(nn.Module):
    def __init__(self, latent_dim,nl):
        super(Encoder, self).__init__()
        self.nl=nl
        self.conv1 = nn.Conv1d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv1d(16, 8, 3, padding=1)
        self.conv3 = nn.Conv1d(8, 8, 3, padding=1)
        self.conv4 = nn.Conv1d(8, 4, 3, padding=1)
        self.fc1 = nn.Linear(4*nl, latent_dim*2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        nl=self.nl
        #print(x.shape)
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool1d(x, 2)
        #print(x.size())
        x = x.view(-1, 4*nl)
        x = self.fc1(x)
        return x

# define decoder in pytorch
class Decoder(nn.Module):
    def __init__(self, latent_dim,nl):
        super(Decoder, self).__init__()
        self.nl=nl
        self.fc1 = nn.Linear(latent_dim, nl*4)
        self.conv1 = nn.ConvTranspose1d(4, 8, 3, padding=1)
        self.conv2 = nn.ConvTranspose1d(8, 8, 3, padding=1)
        self.conv3 = nn.ConvTranspose1d(8, 16, 3, padding=1)
        self.conv4 = nn.ConvTranspose1d(16, 3, 3, padding=1)
    def forward(self, x):
        nl=self.nl
        x = F.relu(self.fc1(x))
        x = x.view(-1, 4, self.nl)
        x = F.relu(self.conv1(x))
        x = F.interpolate(x, scale_factor=2)
        x = F.relu(self.conv2(x))
        x = F.interpolate(x, scale_factor=2)
        x = F.relu(self.conv3(x))
        x = F.interpolate(x, scale_factor=2)
        x = self.conv4(x)
        x = F.interpolate(x, scale_factor=2)
        return x

# define variational autoencoder in pytorch
class VAE(nn.Module):
    def __init__(self, latent_dim,nl):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_dim,nl)
        self.decoder = Decoder(latent_dim,nl)

    def forward(self, x):
        #print('encoder')
        x = self.encoder(x)
        #print('endcoder done')
        z_mean = x[:, 0:latent_dim]
        z_log_var = x[:, latent_dim:]
        #z = self.reparameterization(z_mean, z_log_var)
        x = self.decoder(z_mean)
        return x, z_mean, z_log_var

    def reparameterization(self, z_mean, z_log_var):
        std = torch.exp(z_log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(z_mean)

latent_dim=7


In [4]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Open the file cm1out.nc and read the variables
ncfile = nc.Dataset('/Users/mgrecu/CM1/cm1r21.0/run/cm1out.nc')
u = ncfile.variables['u'][:]
v = ncfile.variables['v'][:]
w = ncfile.variables['w'][:]
dbz=ncfile.variables['dbz'][:]
ncfile.close()
a=np.nonzero(dbz[-1,0,:,:]>0)


IndentationError: expected an indented block (2982523841.py, line 17)

In [54]:
zL=[]
for it in range(-3,0):
    a=np.nonzero(dbz[-1,0,:,:]>0)
    for i in range(len(a[0])):
        if a[1][i]>0 and a[1][i]<dbz.shape[3]-1 and a[0][i]>0 and a[0][i]<dbz.shape[2]-1:
            zL.append(dbz[-1,:,a[0][i]-1:a[0][i]+2,a[1][i]])
        

#i=0
print(len(zL))
zL=np.array(zL)
zL[zL<0]=0
ss=zL.shape
zL+=np.random.normal(0,0.5,ss)

16782


In [118]:
#import standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# fit and transform in one step
zL=zL[:,:64,:]
z_scaled0 = scaler.fit_transform(zL[:,:,0])
z_scaled1 = scaler.transform(zL[:,:,1])
z_scaled2 = scaler.transform(zL[:,:,2])
z_scaled=np.zeros((zL.shape[0],zL.shape[2],zL.shape[1]))
z_scaled[:,0,:]=z_scaled0
z_scaled[:,1,:]=z_scaled1
z_scaled[:,2,:]=z_scaled2


In [117]:
print(zL.shape)

(16782, 64, 3)


In [119]:
#print(z_scaled[:,0,:].mean(axis=0),z_scaled[:,0,:].std(axis=0))
# import test train split
from sklearn.model_selection import train_test_split
# split data into train and test sets
train_x, test_x = train_test_split(z_scaled0, test_size=0.2, random_state=42)


In [112]:

encoder=Encoder(latent_dim,4)
decoder=Decoder(latent_dim,4)
vae=VAE(latent_dim,4)
x=torch.randn(10,3,64)
xp=vae(x)
# train the the variational autoencoder model

# include the KL divergence loss
def loss_function(x_hat, x, z_mean, z_log_var):
    recon_loss = criterion(x_hat, x)
    #recon_loss=((x - x_hat)**2).sum()
    
    #kl_div = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp())
    #kl_div = ((z_log_var.exp())**2 + z_mean**2 - z_log_var - 1/2).sum()
    return recon_loss #+ kl_div

x_data = torch.randn(1000,3,64)
print(x_data.dtype)
# make a custom dataloader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)
        
#print(xp)

torch.float32


In [146]:
class EncoderD(nn.Module):
    def __init__(self, latent_dim, ninp, nh1, nh2):
        super(EncoderD, self).__init__()
        self.fc1 = nn.Linear(ninp, nh1)
        self.fc2 = nn.Linear(nh1, nh2)
        self.fc3 = nn.Linear(nh2, latent_dim*2)
        self.nh1=nh1
        self.nh2=nh2

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# define decoder in pytorch
class DecoderD(nn.Module):
    def __init__(self, latent_dim, nh1, nh2, nout):
        super(DecoderD, self).__init__()
        self.fc1 = nn.Linear(latent_dim, nh1)
        self.fc2 = nn.Linear(nh1, nh2)
        self.fc3 = nn.Linear(nh2, nout)
        self.nh1=nh1
        self.nh2=nh2
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# define variational autoencoder in pytorch
class VAED(nn.Module):
    def __init__(self, latent_dim,ninp,nh1,nh2,nout):
        super(VAED, self).__init__()
        self.encoder = EncoderD(latent_dim,ninp,nh1,nh2)
        self.decoder = DecoderD(latent_dim,nh1,nh2,nout)

    def forward(self, x):
        x = self.encoder(x)
        z_mean = x[:, 0:latent_dim]
        z_log_var = x[:, latent_dim:]
        #z = self.reparameterization(z_mean, z_log_var)
        x = self.decoder(z_mean)
        return x, z_mean, z_log_var

    def reparameterization(self, z_mean, z_log_var):
        std = torch.exp(z_log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(z_mean)

latent_dim=5
ninp=50
nout=50
nh1=16
nh2=16
vae_model=VAED(latent_dim,ninp,nh1,nh2,nout)
optimizer = optim.Adam(vae_model.parameters(), lr=1e-3)   
criterion = nn.MSELoss()

In [ ]:
train_xd=train_x.astype(np.float32)
print(type(train_xd))
training_dataset = CustomDataset(train_xd[:,:50])
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)

for epoch in range(30):
    optimizer.zero_grad()
    for x in iter(train_loader):
    # sample randomnly a subset from x_data
        #print(x.size(),x.dtype)
        #print(x)
        x_hat, z_mean, z_log_var = vae_model(x)
        #print(x_hat.mean(axis=0))
        loss = criterion(x_hat, x)
        #break
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, 100, loss.item()))
    #break


In [145]:
x_hat, z_mean, z_log_var = vae_model(x)
# get the numpy array from the tensor x_hat
#x_hat = x_hat.detach().numpy()
#print(np.corrcoef([x_hat[:,0,0],x.numpy()[:,0,0]]))
#print(x_hat[:,0,0])
#print(z_mean[:,0])
for k in range(30):
    print(np.corrcoef([x_hat.detach().numpy()[:,k],x.numpy()[:,k]])[0,1])

0.8602614239657664
0.8643947197359401
0.8699664663892035
0.8767191931108047
0.8789432545150696
0.8937184438708254
0.9003053167435612
0.904084247891385
0.8993322769979354
0.902101997692287
0.8952079678091176
0.9021785745822102
0.9068533981503365
0.9150317236891765
0.8860956770436951
0.8281564393850359
0.8592869288266074
0.8560480768617976
0.8477685795716313
0.8404787634676419
0.8373428611929232
0.8546197609753172
0.8628151535098803
0.8543315205562543
0.8429583547070253
0.839762097170153
0.8129037890019358
0.8292906441885131
0.844491475657399
0.8412174301215888
